In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from scipy.io import loadmat
import scipy
#import hdf5storage
import csv
from tqdm import trange

import glob

Autosaving every 180 seconds


In [2]:
#  
def get_animal_ids(frame,
                  chain_ids,
                  root_dir,
                  vals,
                  classes,
                  comments=False):
    #frame = 988
    #print ("FRAME: ", frame)
    #print ("classes: ", classes.shape)
    
    dirs = chain_ids[frame]

    if comments:
        print ("DIRS: ", dirs)
    
    animal_ids = []
    animal_ids2 = []
    threshold_purity1 = 0.5
    threshold_purity2 = 0.7

    # HUNGARIAN ALGORITHM BASED SEARCH FOR MOST CORRECT IDS
    best_ids_cts = np.zeros((5,5),'float32')
    animals3 = []
    for ctr_dir, dir_ in enumerate(dirs):
        #print ("2nd Loop: ", dir_)
        if np.isnan(dir_)==False:
            # find the classifier values for that segment
            idx = np.where(vals==dir_)[0]

            # find predicted id
            fnames = np.sort(glob.glob(root_dir+str(int(dir_))+'/*.npy',recursive = False))

            ctr_chain = 0
            for fname in fnames:
                if ('frame_'+str(frame).zfill(7)) in fname:
                     break 
                ctr_chain+=1
        
            animal_id3 = classes[idx][ctr_chain]
            animals3.append(animal_id3)
            
            idx5 = np.where(classes[idx]<1E10)[0]
            classes_local = classes[idx[idx5]]
            
            classes_unique = np.unique(classes_local, return_counts=True)
            #print ("best id: ", best_id)
            
            best_ids_cts[ctr_dir, classes_unique[0]]=classes_unique[1]/idx.shape[0]
        else:
            animals3.append(None)
    
    # FIRST LOOP TO CHECK ANY GREAT MATCHES AND ZERO OUT EVERYTHING ELSE
    for k1 in range(5):
        temp = best_ids_cts[k1]
        max_val = np.max(temp)
        if max_val>threshold_purity2:
            idx8 = np.argmax(temp)
            best_ids_cts[:,idx8]=0
            best_ids_cts[k1,idx8]=max_val
    if comments:
        print (best_ids_cts)
    # RENORMALIZE PROBABILITIES AFTER ZEROING OUT
    for k1 in range(5):
        sum_ = best_ids_cts[k1].sum()
        if sum_>0:
            best_ids_cts[k1]*= 1./sum_
    if comments:
        print (best_ids_cts)    
        
    # FINAL LOOP TO GRAB WHATEVER IS LEFT
    for k1 in range(5):
        max_val = np.max(best_ids_cts[k1])
        if max_val==0: # or max_val<threshold_purity1:
            animal_ids.append(None)
            animal_ids2.append(animals3[k1])
            continue
        else:
            idx9 = np.argmax(best_ids_cts[k1])
            #print ("idx9: ", idx9, classes_unique[0])
            animal_ids.append(idx9)
            animal_ids2.append(animals3[k1])
            
            best_ids_cts[:,idx9]= 0
            # must zero out all the others
            
    if comments:
        print (best_ids_cts)           
    return animal_ids, animal_ids2, dirs

def get_animal_ids_function(fname_classification,
                   fname_traces_inferences,
                   chain_ids,
                   root_dir,
                   comments,
                   n_networks):
    
    fname_out = os.path.split(fname_classification)[0]+"/animal_arrays.npy"
    if os.path.exists(fname_out)==False:

        data = np.load(fname_classification, allow_pickle=True)
        vals = data['vals']
        frames = np.hstack(data['frames_array'])
        print ("# of lableed frames: ", vals.shape)
        classes = data['classes']

        reassembled = np.load(fname_traces_inferences)

        #reassembled = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/pickle/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz')
        tracesx_re = reassembled['tracesx'].T
        tracesy_re = reassembled['tracesy'].T
        print ("Lenght of tracessx: ", tracesx_re.shape)

        start = 1
        end = tracesx_re.shape[0]

        # INITIALIZE ARRAYS TO BE SAVED
        animal_ids_hungarian = np.zeros((tracesx_re.shape[0], n_networks), 'float32')
        animal_arrays = np.zeros((n_networks, tracesx_re.shape[0], 2), 'float32')
        for n in trange(start,end, 1):
            #ret, frame = original_vid.read()

            # animal_ids is the hungarian output;
            # animal_ids is the CNN output
            animal_ids, animal_ids2, dirs = get_animal_ids(n,
                                                        chain_ids,
                                                        root_dir,
                                                        vals,
                                                        classes,
                                                        comments)

            #print ("animal_ids: ", animal_ids)
            for k in range(0, 14*n_networks,14):
                y = tracesx_re[n,k:k+14]
                x = tracesy_re[n,k:k+14]

                idx = np.where(x!=0)[0]
                x = x[idx]
                y = y[idx]
                if x.shape[0]==0:
                    x_ave = None
                    y_ave = None
                else:
                    x_ave = np.nanmean(x)
                    y_ave = np.nanmean(y)

                if animal_ids[k//14] is None:
                    continue
                animal_arrays[animal_ids[k//14], n] = x_ave, y_ave

            animal_ids_hungarian[n]=animal_ids 

        np.save(fname_out, animal_arrays)
        np.save(fname_out[:-4]+"_animal_ids_hungarian.npy", animal_ids_hungarian)
    else:
        animal_arrays = np.load(fname_out)
        animal_ids_hungarian = np.load(fname_out[:-4]+"_animal_ids_hungarian.npy")

    print ("DONE making animal_arrays: ", animal_arrays.shape)

    return animal_arrays, animal_ids_hungarian

def get_single_feature_traces(fname_classification,
                              fname_traces_inferences,
                              animal_ids_hungarian,
                              n_networks,
                              features):
    
    fname_out = os.path.split(fname_classification)[0] + "/locs_array.npy"
    if os.path.exists(fname_out)==False:
    
        reassembled = np.load(fname_traces_inferences)

        #reassembled = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/pickle/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz')
        tracesx_re = reassembled['tracesx'].T
        tracesy_re = reassembled['tracesy'].T

        locs_array = np.zeros((n_networks, animal_ids_hungarian.shape[0],2),'float32')+np.nan
        
        for p in ids:
            # Plot spine1 location for each animal
            #locs = np.zeros(animal_ids_hungarian.shape[0],2,'float32')
            for k in range(1, animal_ids_hungarian.shape[0]):
                idxb = np.where(animal_ids_hungarian[k]==p)[0]
                if idxb.shape[0]>0:
                    tempx = tracesx_re[k,features+idxb*14]
                    #print ("tempx: ", tempx.shape)
                    tempy = tracesy_re[k, features+idxb*14]
                    locs_array[p,k]= tempx, tempy

        np.save(fname_out, locs_array)
    
    else:
        locs_array = np.load(fname_out)
        
    return locs_array


def cleanup_traces(locs_array, 
                  animal_id,
                  velocity_max,
                  velocity_min,
                  velocity_rel_max,
                  min_seg_len):
    
    data = locs_array[animal_id].copy()
    
    # REMOVE GARBAGE / FAILED PREDICTION
    idx = np.where(data[:,0]==0.)[0]
    #print (" ZERO REMOVAL: ", idx.shape)
    data[idx,0]=np.nan
    data[idx,1]=np.nan

    # DEL SHORT SEGS 
    idx = np.where(np.isnan(data[:,0])==True)[0]
    diffs = idx[1:]-idx[:-1]
    idx2 = np.where(np.logical_and(diffs>1, diffs<min_seg_len))[0]
    #print ("short segs deletion: ", idx2.shape)

    # loop over short segs and remove them
    for id_ in idx2:
        data[idx[id_]:idx[id_+1]]=np.nan

    # VELOCITIES
    if True:

        if False:
            plt.scatter(np.arange(data.shape[0])/25, 
                    data.sum(1), color='black')
            
        #temp = temp1[1:]-temp1[:-1]

        locs_temp = np.sqrt(data[:,0]**2+data[:,1]**2)
        velocity = locs_temp[1:]-locs_temp[:-1]

        # absolute velocity delete
        idx = np.where(np.abs(velocity)>velocity_max)[0]
        data[idx]=np.nan

        # relative velocity delete
        idx = np.where(np.logical_and(np.abs(velocity[1:])>np.abs(velocity[:-1])*velocity_rel_max,
                                     np.abs(velocity[1:])>velocity_min))[0]

        #print ("relative vellcity deletions ", idx.shape)
        data[idx]=np.nan   
        
        idx = np.where(np.logical_and(np.abs(velocity[1:])*velocity_rel_max<np.abs(velocity[:-1]),
                                     np.abs(velocity[1:])>velocity_min))[0]

        #print ("relative vellcity deletions ", idx.shape)
        data[idx]=np.nan   

        

    # DEL SHORT SEGS 
    idx = np.where(np.isnan(data[:,0])==True)[0]
    diffs = idx[1:]-idx[:-1]
    idx2 = np.where(np.logical_and(diffs>1, diffs<min_seg_len))[0]
    #print ("short segs deletion: ", idx2.shape)

    # loop over short segs and remove them
    for id_ in idx2:
        data[idx[id_]:idx[id_+1]]=np.nan
        
    locs_array[animal_id] = data

    return locs_array

def connect_CSUA(locs_array,
                       animal_id,
                       max_jump, 
                       max_jump_intra_animal,     
                       min_nearby_animal_dist):
    
    animal_ids = np.arange(4)
    
    data = locs_array[animal_id].copy()
    #print ("Data: ", data.shape)
    
    
    idx = np.where(np.isnan(data[:,0])==False)[0]
   # print ("interpo: ", idx.shape)
   # print (idx[:10])
    diffs= idx[1:]-idx[:-1]
    idx2 = np.where(diffs>1)[0]
    #print (idx[idx2[:10]])
    
    idxnan = np.where(np.isnan(data[:,0])==True)[0]
    diffsnan = idxnan[1:]-idxnan[:-1]
    idx2nan = np.where(diffsnan>1)[0]
    
    for k in range(1,idx2.shape[0]-1,1):
        start = idx[idx2[k]]
        duration = diffs[idx2[k]]
        end = idxnan[idx2nan[k+1]]+1
        
        dist = np.sqrt((data[start,0]-data[end,0])**2+
                       (data[start,1]-data[end,1])**2)
        
        # CHECK MIN DIST; SET THIS TO CONSERVATIVE VALUE
        if dist <= max_jump:
                
            #print (data[start,0], data[end,0], duration)
            data[start:end,0]= np.linspace(data[start,0], data[end,0], duration)
            data[start:end,1]= np.linspace(data[start,1], data[end,1], duration)
            
        # CHECK MORE BROADLY ENSURE NO OTHER ANIMAL NEARBY
        elif dist <= max_jump_intra_animal:
            for a in animal_ids:
                if a == animal_id:
                    continue
                    #dists_start.append(1E10)
                    #dists_end.append(1E10)
                    
                near_anim = np.sqrt((locs_array[a][start,0]-locs_array[animal_id][start,0])**2+
                                      (locs_array[a][start,1]-locs_array[animal_id][start,1])**2)
                
                if np.min(near_anim)<min_nearby_animal_dist:
                    continue
                    
                near_anim = np.sqrt((locs_array[a][end,0]-locs_array[animal_id][end,0])**2+
                                      (locs_array[a][end,1]-locs_array[animal_id][end,1])**2)
                
                if np.min(near_anim)<min_nearby_animal_dist:
                    continue

            #print (data[start,0], data[end,0], duration)
            data[start:end,0]= np.linspace(data[start,0], data[end,0], duration)
            data[start:end,1]= np.linspace(data[start,1], data[end,1], duration)
            
    locs_array[animal_id] = data
    
    return locs_array



def swap_traces(locs_array, 
                animal_id,
                animal_ids):
    '''  Function that tries to correct obvious swaps
    '''
    
    # GRAB CSUAs FOR ANIMAL AND SEE IF BETTER MATCH WITH OTHER STARTS/ENDS
    # GRAB CSUAs from main animal
    idx = np.where(np.isnan(locs_array[animal_id])==False)[0]
    diffs = idx[1:]-idx[:-1]
    idx_end = np.where(diffs>1)[0]
    
    idxnan = np.where(np.isnan(locs_array[animal_id])==True)[0]
    diffs = idxnan[1:]-idxnan[:-1]
    idx_start = np.where(diffs>1)[0]
    
    CSUA = np.array([idxnan[idx_start],
                    idx[idx_end]]).T
    print ("animal id: ", animal_id, 
           "CSUA start: ", CSUA)
    
    # FIND WHICH CSUAS HAVE LARGE JUMPS FROM NEIGHBOURS
    
    
#     # FIND CSUAs for other animals
#     for a in animal_ids:
#         idx = np.where(np.isnan(locs_array[a])==False)[0]
#         diffs = idx[1:]-idx[:-1]
#         idx2 = np.where(diffs>1)[0]    
    
    
    return locs_array



def cleanup(fname_classification, 
            animal_arrays,
            animal_ids,
            locs_array,
            velocity_max = 200,
            velocity_min = 10.,
            velocity_rel_max = 7.,
            min_seg_len = 5,
            max_jump = 100, # SAFE DISTANCE TO MERGE SPLIT 
            max_jump_intra_animal = 500, # DISTANCE TO MERGE IF NO OTHER ANIMALS "NEARBY"
            min_nearby_animal_dist = 30, # DISTANCE DEFINING "NEARBY"
            separation_dist = 100, # Distance under which labels can no longer change location
           ):

    for animal_id in animal_ids:
        #print (locs_array[animal_id].shape)

        ###################################################
        ################## CLEANUP TRACES #################
        ###################################################
        if True:
            locs_array = cleanup_traces(locs_array,
                              animal_id,
                              velocity_min,
                              velocity_min,
                              velocity_rel_max,
                              min_seg_len)

        ###################################################
        ############# SWAP TRACES - NOT IMPLEMENTED #######
        ###################################################
    #     locs_array = swap_traces(locs_array,
    #                              animal_id,
    #                              animal_ids)

        ###################################################
        ############## CONNECTS CSUAS  ####################
        ###################################################
        if True:
            # REPEAT INTERPOLATION X TIMES
            for k in range(5):
                locs_array = connect_CSUA(locs_array,
                                                animal_id,
                                                max_jump,
                                                max_jump_intra_animal,     
                                                min_nearby_animal_dist)
        ###################################################
        ################# INTERPOLATE TRACES ##############
        ###################################################
        if True:
            # Fix x,y positions for missing data to last value in time
            # TODO: Try to recover missing data by looking at 
            #  raw heatmaps or peaks outputed by DLC
            locs_array = interpolate_traces(locs_array,
                                            animal_id)
        
        
        ###################################################
        ######## STOPS TRACES FROM JUMPING/JITTER #########
        ###################################################
        locs_array = stop_traces_jumps(locs_array, separation_dist)

            

    return locs_array 

def interpolate_traces(locs_array,
                       animal_id):
    
    ''' Replace all np.nans by the last good value
    '''
        
    data = locs_array[animal_id].copy()
    #print ("Data: ", data.shape)
    
    idx = np.where(np.isnan(data)==True)[0]
    for id_ in idx:
        data[id_]=data[id_-1]
    
    # go backwoards in time to ensure beginning of time also has non np.nan vals
    data = data[::-1]
    idx = np.where(np.isnan(data)==True)[0]
    for id_ in idx:
        data[id_]=data[id_-1]

    data = data[::-1]
    locs_array[animal_id] = data
    
    return locs_array

def stop_traces_jumps(locs_array, separation_dist):
    ''' Function that freezes traces once an animal gets near another
    '''
    
    # COMPUTE INTER-ANIMAL DISTANCES AT EVERY POINT:
    
    for a in range(4):
        
        locs_a = locs_array[a]
        
        for aa in range(4):
            if a==aa:
                continue
            locs_aa = locs_array[aa]
            
            dists = np.sqrt((locs_a[:,0]-locs_aa[:,0])**2+
                            (locs_a[:,1]-locs_aa[:,1])**2)
            
            idx = np.where(dists<separation_dist)[0]
            # search forward in time and fix to last value if too close 
            for id_ in idx:
                locs_a[id_]=locs_a[id_-1]

    
    return locs_array

In [10]:
#######################################################################
###### COMBINE CNN OUTPUT AND SLEAP OUTPUT + HUNGARIAN ALGORITHM ######
#######################################################################

# Classifier output for CSUA; using 100 x 100 pixel images;
fname_classification = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/classification_output.npz'
                        #/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/classification_output.npz

# Notebook 11 - CC based image extraction generates chain_id file 
fname_chain_ids = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences_0_89988_chain_id.npy'
chain_ids = np.load(fname_chain_ids)
print (chain_ids.shape)

# CSUA image locations; required for hungarian; may wish to simplifiy;
root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/'

# Traces post-CC step
fname_traces_inferences = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences_0_89988.npz'
                          #'/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences_0_89988.npz'

comments=False
n_networks = 5
animal_ids, animal_ids_hungarian = get_animal_ids_function(fname_classification,
                                                  fname_traces_inferences,
                                                   chain_ids,
                                                   root_dir,
                                                   comments,
                                                   n_networks)
print ("DONE computing hungarian ID assignments")

(89988, 5)
DONE making animal_arrays:  (5, 89988, 2)
DONE computing hungarian ID assignments


In [13]:
##############################################################
####### COMPUTE SINGLE FEATURE TRACES POST HUNGARIAN #########
##############################################################
features = np.array(6, 'int32') # spine_1
ids = np.arange(5)
#ids=[0]

locs_array = get_single_feature_traces(fname_classification,
                                      fname_traces_inferences,
                                      animal_ids_hungarian,
                                      n_networks,
                                      features)

print ("DONE with single feature post huganrian", locs_array.shape)

DONE with single feature post huganrian (5, 89988, 2)


In [14]:
#################################################################
###### CLEANUP: Del short segs; connect safe segs ###############
#################################################################
#clrs = ['red','blue','cyan','green','yellow']
clrs = ['blue','red','yellow','green','yellow']

    
#print (locs_array.shape)
animal_arrays = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/animal_arrays.npy')
#print (animal_arrays.shape)
animal_arrays = locs_array.copy()

animal_ids = np.arange(5)
locs_array_clean = cleanup(fname_classification, 
                            animal_arrays,
                            animal_ids,
                            locs_array)
            
np.save(os.path.split(fname_classification)[0]+'/locs_array_clean.npy', 
                       locs_array_clean)

print ("DONE Post-processing traces - GO TO Notebook 15_behaviour_analysis")

/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ipykernel_launcher.py:235: RuntimeWarning: invalid value encountered in greater
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ipykernel_launcher.py:239: RuntimeWarning: invalid value encountered in greater
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ipykernel_launcher.py:240: RuntimeWarning: invalid value encountered in greater
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ipykernel_launcher.py:245: RuntimeWarning: invalid value encountered in less
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ipykernel_launcher.py:246: RuntimeWarning: invalid value encountered in greater
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ipykernel_launcher.py:474: RuntimeWarning: invalid value encountered in less


DONE Post-processing traces - GO TO Notebook 15_behaviour_analysis
